In [13]:
import pandas as pd
import numpy as np

In [14]:
# import train.csv
url_train = "https://raw.githubusercontent.com/naufaldi-fir/adoption-prediction-project/main/Data%20Set/train.csv"
train = pd.read_csv(url_train)

# import test.csv
url_test = "https://raw.githubusercontent.com/naufaldi-fir/adoption-prediction-project/main/Data%20Set/test.csv"
test = pd.read_csv(url_test)

In [15]:
print(train.head())

   Type         Name  Age  Breed1  Breed2  Gender  Color1  Color2  Color3  \
0     2       Nibble    3     299       0       1       1       7       0   
1     2  No Name Yet    1     265       0       1       1       2       0   
2     1       Brisco    1     307       0       1       2       7       0   
3     1         Miko    4     307       0       2       1       2       0   
4     1       Hunter    1     307       0       1       1       0       0   

   MaturitySize  ...  Health  Quantity  Fee  State  \
0             1  ...       1         1  100  41326   
1             2  ...       1         1    0  41401   
2             2  ...       1         1    0  41326   
3             2  ...       1         1  150  41401   
4             2  ...       1         1    0  41326   

                          RescuerID  VideoAmt  \
0  8480853f516546f6cf33aa88cd76c379         0   
1  3082c7125d8fb66f7dd4bff4192c8b14         0   
2  fa90fa5b1ee11c86938398b60abc32cb         0   
3  9238e4f44c71a

### One hot encoding for color in train.csv

In [16]:
# drop all columns except color
color = train.loc[:,["Color1","Color2","Color3"]]
color.head()

,Color1,Color2,Color3
0,1,7,0
1,1,2,0
2,2,7,0
3,1,2,0
4,1,0,0


In [17]:
# because it doesn't make a lot of sense to sort the color in Color 1,2 and 3. The table will be 
# transformed to binary data using one hot encoding
dum_color1 = pd.get_dummies(color.Color1,prefix = "ColorID",drop_first =True)
dum_color2 = pd.get_dummies(color.Color2,prefix = "ColorID",drop_first = True)
dum_color3 = pd.get_dummies(color.Color3,prefix = "ColorID", drop_first = True)

In [18]:
all_color = [dum_color1,dum_color2,dum_color3]

In [19]:
# define print color function
def print_color(list):
    i = 1
    for x in list:
        print(f"Dummy Color {i}")
        print(x.head())
        print(f"Shape of the table {x.shape}")
        i = i + 1

In [20]:
# print all color
# print all one hot encoding from get_dummies function => 
# The dummies have different shape and attribute for each table
print_color(all_color)

Dummy Color 1
   ColorID_2  ColorID_3  ColorID_4  ColorID_5  ColorID_6  ColorID_7
0          0          0          0          0          0          0
1          0          0          0          0          0          0
2          1          0          0          0          0          0
3          0          0          0          0          0          0
4          0          0          0          0          0          0
Shape of the table (14993, 6)
Dummy Color 2
   ColorID_2  ColorID_3  ColorID_4  ColorID_5  ColorID_6  ColorID_7
0          0          0          0          0          0          1
1          1          0          0          0          0          0
2          0          0          0          0          0          1
3          1          0          0          0          0          0
4          0          0          0          0          0          0
Shape of the table (14993, 6)
Dummy Color 3
   ColorID_3  ColorID_4  ColorID_5  ColorID_6  ColorID_7
0          0          0  

The difference between dummy color 1 and dummy color 2 are in the existence of attribute ColorID_0 and ColorID_1

In [21]:
# changing the attribute and shape of the table so we can sum all the tables
dum_color3["ColorID_2"] = 0

In [22]:
# print all color after edit
print_color(all_color)

Dummy Color 1
   ColorID_2  ColorID_3  ColorID_4  ColorID_5  ColorID_6  ColorID_7
0          0          0          0          0          0          0
1          0          0          0          0          0          0
2          1          0          0          0          0          0
3          0          0          0          0          0          0
4          0          0          0          0          0          0
Shape of the table (14993, 6)
Dummy Color 2
   ColorID_2  ColorID_3  ColorID_4  ColorID_5  ColorID_6  ColorID_7
0          0          0          0          0          0          1
1          1          0          0          0          0          0
2          0          0          0          0          0          1
3          1          0          0          0          0          0
4          0          0          0          0          0          0
Shape of the table (14993, 6)
Dummy Color 3
   ColorID_3  ColorID_4  ColorID_5  ColorID_6  ColorID_7  ColorID_2
0          0  

all dummies have now same attributes and shape

In [23]:
# adding all the data
dum_color = dum_color1 + dum_color2 + dum_color3
dum_color.head()

,ColorID_2,ColorID_3,ColorID_4,ColorID_5,ColorID_6,ColorID_7
0,0,0,0,0,0,1
1,1,0,0,0,0,0
2,1,0,0,0,0,1
3,1,0,0,0,0,0
4,0,0,0,0,0,0


In [25]:
# check one hot encoding with train data
pd.concat([color, dum_color],axis = 1).iloc[30:40]

,Color1,Color2,Color3,ColorID_2,ColorID_3,ColorID_4,ColorID_5,ColorID_6,ColorID_7
30,2,7,0,1,0,0,0,0,1
31,1,0,0,0,0,0,0,0,0
32,5,7,0,0,0,0,1,0,1
33,1,6,7,0,0,0,0,1,1
34,1,7,0,0,0,0,0,0,1
35,6,0,0,0,0,0,0,1,0
36,1,4,0,0,0,1,0,0,0
37,2,0,0,1,0,0,0,0,0
38,1,0,0,0,0,0,0,0,0
39,5,0,0,0,0,0,1,0,0


 one hot encoding success

In [31]:
df_1= pd.DataFrame({"a": [1],"b":[5]})
df_2 = pd.DataFrame({"b": [100],"a":[200]})
df = df_1 + df_2
print(df)

     a    b
0  201  105


# Using Classification model KNN on color

## A. Preparing data

### Data Train
divide the train data in train and test

### Data Test
import test data, do the one hot encoding one moretime
pipeline? or using OneHotEncoder from scikit

## B. KNN and SVC Implementation
make the knn and svc classifier
using cross validation to evaluate the both model in train data

## make a report for test data